In [1]:
import numpy as np

In [2]:
def taking_input():
  A1= input("Enter the coefficients of eqaution {}:  ".format(1)).split()
  # print(len(A1))
  ls = []
  ls.append(A1)
  for i in range(len(A1)-1):
    ls.append(input("Enter the coefficients of eqaution {}: ".format(i+2)).split())

  A = np.array(ls).astype(float)
  b = np.array(input("Enter the constants: ").split()).astype(float)
  return A,b


## (a) Gauss Elimination without pivoting

In [3]:
def forward_elimination(A, b):
    for row in range(0, A.shape[0]-1):
        for i in range(row+1, A.shape[0]):
            factor = A[i,row] / A[row,row]
            for j in range(row, A.shape[0]):
                A[i,j] = A[i,j] - factor * A[row,j]
            b[i] = b[i] - factor * b[row]

        # print('A = \n%s and b = %s' % (A,b))
    return A, b

def back_substitution(a, b):
    x = np.zeros((a.shape[0],1))
    x[a.shape[0]-1] = b[a.shape[0]-1] / a[a.shape[0]-1, a.shape[0]-1]
    for row in range(a.shape[0]-2, -1, -1):
        sums = b[row]
        for j in range(row+1, a.shape[0]):
            sums = sums - a[row,j] * x[j]
        x[row] = sums / a[row,row]
    return x

def gauss_without_pivoting(A, b):
    if any(np.diag(A)==0):
        raise ZeroDivisionError(('Division by zero will occur; '
                                  'pivoting currently not supported'))

    A, b = forward_elimination(A, b)
    return back_substitution(A, b)

2
## (b) Gauss Elimination with pivoting

In [4]:
def gep(A,b):
  n = len(A)
  M = A

  i = 0
  for x in M:
   np.append(x,b[i])
   x.append(b[i])
   i += 1
  for k in range(n):
   for i in range(k,n):
     if abs(M[i][k]) > abs(M[k][k]):
        M[k], M[i] = M[i],M[k]
     else:
        pass

   for j in range(k+1,n):
       q = float(M[j][k]) / M[k][k]
       for m in range(k, n+1):
          M[j][m] -=  q * M[k][m]

  x = [0 for i in range(n)]

  x[n-1] =float(M[n-1][n])/M[n-1][n-1]
  for i in range (n-1,-1,-1):
    z = 0
    for j in range(i+1,n):
        z = z  + float(M[i][j])*x[j]
    x[i] = float(M[i][n] - z)/M[i][i]
  return x



## (c) Gauss Elimination with pivoting and scaling


In [5]:
import numpy as np
def gauss_pivot_scaling(a,b,tol=1.0e-12):

    a = np.copy(a)
    b = np.copy(b)
    n = len(b)
    assert (np.all(np.shape(a) ==(n,n))) 
    
    s = np.zeros(n)
    for i in range(n):
        s[i] = max(np.abs(a[i,:]))
    for k in range(0, n-1): #pivot row
        # Row interchange, if needed
        p = np.argmax(np.abs(a[k:n,k])/s[k:n]) 
        if abs(a[p,k]) < tol: 
            raise Exception("Matrix is singular")
        if p != k: 
            a[[k,p+k],:] = a[[p+k, k],:]
            b[k],b[p+k] = b[p+k],b[k] 
            s[k],s[p+k] = s[p+k],s[k]

        
        for i in range(k+1,n):
            if a[i,k] != 0.0: 
                lam = a [i,k]/a[k,k] 
                a[i,k:n] = a[i,k:n] - lam*a[k,k:n]
                b[i] = b[i] - lam*b[k]

    if abs(a[n-1,n-1]) < tol: 
        raise Exception("Matrix is singular")

    x = np.zeros_like(b)
    x[n-1] = b[n-1]/a[n-1,n-1]
    for k in range(n-2,-1,-1):
        x[k] = (b[k] - np.dot(a[k,k+1:n],x[k+1:n]))/a[k,k]
    return x




## (d) LU without pivoting



In [6]:
def lu_without_pivoting(A):
    n = A.shape[0]
    U = A.copy()
    L = np.eye(n, dtype=np.double)
    for i in range(n):
        factor = U[i+1:, i] / U[i, i]
        L[i+1:, i] = factor
        U[i+1:] -= factor[:, np.newaxis] * U[i]  
    return L, U

def forward_substitution(L, b):
    n = L.shape[0]
    y = np.zeros_like(b, dtype=np.double);
    y[0] = b[0] / L[0, 0]
    for i in range(1, n):
        y[i] = (b[i] - np.dot(L[i,:i], y[:i])) / L[i,i]
    return y

def back_substitution(U, y):
    n = U.shape[0]
    x = np.zeros_like(y, dtype=np.double);
    x[-1] = y[-1] / U[-1, -1]
    for i in range(n-2, -1, -1):
        x[i] = (y[i] - np.dot(U[i,i:], x[i:])) / U[i,i]
    return x

def lu_without_pivoting_solve(A, b):
    L, U = lu_without_pivoting(A)
    print("L = " + str(L) + "\n")
    print("U = " + str(U) + "\n")
    y = forward_substitution(np.array(L), b)
    return back_substitution(np.array(U), y)




## (e) LU with pivoting

In [7]:
def LU(A):
	n = len(A)
	b = [0 for i in range(n)]
	for i in range(0,n):
		b[i]=A[i]*n

	L_p = [[0 for i in range(n)] for i in range(n)]
	for i in range(0,n):
		L_p[i][i] = 1

	U_p = [[0 for i in range(0,n)] for i in range(n)]
	for i in range(0,n):
		for j in range(0,n):
			U_p[i][j] = A[i][j]

	n = len(U_p)

	for i in range(0,n):
		maxElem = abs(U_p[i][i])
		maxRow = i
		for k in range(i+1, n):
			if(abs(U_p[k][i]) > maxElem):
				maxElem = abs(U_p[k][i])
				maxRow = k

		for k in range(i, n):
			tmp=U_p[maxRow][k]
			U_p[maxRow][k]=U_p[i][k]
			U_p[i][k]=tmp

		for k in range(i+1,n):
			c = -U_p[k][i]/float(U_p[i][i])
			L_p[k][i] = c 
			for j in range(i, n):
				U_p[k][j] += c*U_p[i][j] 

		for k in range(i+1, n):
			U_p[k][i]=0
	return L_p, U_p

def forward_substitution(L, b):
    n = L.shape[0]
    y = np.zeros_like(b, dtype=np.double);
    y[0] = b[0] / L[0, 0]
    for i in range(1, n):
        y[i] = (b[i] - np.dot(L[i,:i], y[:i])) / L[i,i]
        
    return y

def back_substitution(U, y):
    n = U.shape[0]
    x = np.zeros_like(y, dtype=np.double)
    x[-1] = y[-1] / U[-1, -1]
    for i in range(n-2, -1, -1):
        x[i] = (y[i] - np.dot(U[i,i:], x[i:])) / U[i,i]
    return x
def lu_with_pivoting_solve(A, b):
  L_p, U_p = LU(A)
  L_p = np.array(L_p)
  L_p = 2*np.eye(len(A)) -L_p
  U_p = np.array(U_p)
  print("L = " + str(L_p) + "\n")
  print("U = " + str(U_p) + "\n")
  y = forward_substitution(L_p, b)
  return back_substitution(U_p, y)


## (f) Crout Method

In [8]:

def crout(A):
    L = np.matrix([[0.0,0.0,0.0],[0.0,0.0,0.0],[0.0,0.0,0.0]])
    U = np.matrix([[0.0,0.0,0.0],[0.0,0.0,0.0],[0.0,0.0,0.0]])
    n = len(A)
    for z in range(n):
        U[z,z] = 1             
        for j in range(z,n):
            temporary_L = float(A[j,z])
            for k in range(z):
                temporary_L -= L[j,k]*U[k,z]
            L[j,z] = temporary_L
        for j in range(z+1, n):
            temporary_U = float(A[z,j])
            for k in range(z):
                temporary_U -= L[z,k]*U[k,j]
            U[z,j] = temporary_U / L[z,z]
    return (L, U)

def forward_substitution(L, b):
    y = np.full_like(b,0)
    for k in range(len(b)):
        y[k] = b[k]
        for i in range(k):
            y[k] = y[k] - (L[k, i]*y[i])
        y[k] = y[k] / L[k, k]
    return y

def backward_substitution(U, y):
    x = np.full_like(y,0)
    for k in range(len(x), 0, -1):
      x[k-1] = (y[k-1] - np.dot(U[k-1, k:], x[k:])) / U[k-1, k-1]
    return x

def final_crout_solution(A, b):
    L, U = crout(A)
    print("L = " + str(L) + "\n")
    print("U = " + str(U) + "\n")
    
    y = forward_substitution(L,b)
    x = backward_substitution(U,y)
    
    return x

## (g) Cholesky decomposition

In [9]:
import numpy as np
from math import sqrt


def cholesky_decomposition(M):
  

    A = np.copy(M)
    n = A.shape[0]
    R = np.zeros_like(A)

    for k in range(n):
        R[k, k] = sqrt(A[k, k])
        R[k, k + 1:] = A[k, k + 1:] / R[k, k]
        for j in range(k + 1, n):
            A[j, j:] = A[j, j:] - R[k, j] * R[k, j:]

    return R



def forward_substitution(L, b):
    n = L.shape[0]
    y = np.zeros_like(b, dtype=np.double);
    y[0] = b[0] / L[0, 0]
    for i in range(1, n):
        y[i] = (b[i] - np.dot(L[i,:i], y[:i])) / L[i,i]
        
    return y

def back_substitution(U, y):
    n = U.shape[0]
    x = np.zeros_like(y, dtype=np.double);
    x[-1] = y[-1] / U[-1, -1]
    for i in range(n-2, -1, -1):
        x[i] = (y[i] - np.dot(U[i,i:], x[i:])) / U[i,i]        
    return x

def final_cholesky_solution(A, b):
    U = cholesky_decomposition(A)
    L = U.T
    print("L = " + str(L) + "\n")
    print("U = " + str(U) + "\n")
    
    y = forward_substitution(L,b)
    x = backward_substitution(U,y)
    
    return x

In [ ]:
def Question_1(method):
    switcher = {
        1: "Gauss Elimination without pivoting",
        2: "Gauss Elimination with pivoting",
        3: "Gauss Elimination with pivoting and scaling",
        4: "LU without pivoting",
        5: "LU with pivoting",
        6: "Crout Method",
        7:"Cholesky decomposition"
    }


    if switcher.get(method) == "Gauss Elimination without pivoting":
      print("You choose: " + switcher.get(method))
      A,b = taking_input()
      x = gauss_without_pivoting(A,b)
      print(x)
      
      

    elif switcher.get(method) == "Gauss Elimination with pivoting":
      print("You choose: " + switcher.get(method))
      A,b = taking_input()
      x = gep(A.tolist(),b.tolist())
      print(x)


     



    elif switcher.get(method) == "Gauss Elimination with pivoting and scaling":
      print("You choose: " + switcher.get(method))
      A,b = taking_input()
      x = gauss_pivot_scaling(A,b)
      print(x)
      




    elif switcher.get(method) == "LU without pivoting":
      print("You choose: " + switcher.get(method))
      A,b = taking_input()
      x = lu_without_pivoting_solve(A,b)
      print(x)

    elif switcher.get(method) == "LU with pivoting":
      print("You choose: " + switcher.get(method))
      A,b = taking_input()
      x = lu_with_pivoting_solve(A,b)
      print(x)

    elif switcher.get(method) == "Crout Method":
      print("You choose: " + switcher.get(method))
      A,b = taking_input()
      x = final_crout_solution(A,b)
      print(x)
      

    elif switcher.get(method) == "Cholesky decomposition":
      print("You choose: " + switcher.get(method))
      A,b = taking_input()
      x = final_cholesky_solution(A,b)
      print(x)

    else:
      print("Invalid method")

print("Choose the number : ", {
      1: "Gauss Elimination without pivoting",
        2: "Gauss Elimination with pivoting",
        3: "Gauss Elimination with pivoting and scaling",
        4: "LU without pivoting",
        5: "LU with pivoting",
        6: "Crout Method",
        7:"Cholesky decomposition"
    })

# A = np.array([[4, 2, 0], [2, 4, 1], [0, 1., 5]])
# b = np.array([10, 11.5, 5.])
Question_1(int(input()))

Choose the number :  {1: 'Gauss Elimination without pivoting', 2: 'Gauss Elimination with pivoting', 3: 'Gauss Elimination with pivoting and scaling', 4: 'LU without pivoting', 5: 'LU with pivoting', 6: 'Crout Method', 7: 'Cholesky decomposition'}
7
You choose: Cholesky decomposition
Enter the coefficients of eqaution 1:  4 2 0
Enter the coefficients of eqaution 2: 2 4 1
Enter the coefficients of eqaution 3: 0 1 5
Enter the constants: 10 11.5 5
L = [[2.         0.         0.        ]
 [1.         1.73205081 0.        ]
 [0.         0.57735027 2.1602469 ]]

U = [[2.         1.         0.        ]
 [0.         1.73205081 0.57735027]
 [0.         0.         2.1602469 ]]

[1.51785714 1.96428571 0.60714286]
